In [1]:
import os
import os.path
import csv
import time
import PIL
import math
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

from ImageNet import *
from CNN1 import *
from ResNet import *
from utils import *

from pytorchtools import EarlyStopping

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

In [3]:
def topaccuracy(output, target, topk=(5, )):       
    # output.shape (bs, num_classes), target.shape (bs, )
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
    return correct[:5].reshape(-1).float().sum(0, keepdim=True)

In [4]:
data_dir = './data'

workers = 1
batch_size = 128

max_epochs = 100
lr = 0.015

gpu = torch.cuda.is_available()

if gpu:
    print('Training on GPU')
else:
    print('Training on CPU')

Training on GPU


In [5]:
transform1 = transforms.RandomApply([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, shear=45, fillcolor=(255, 0, 0)),
],p=0.5)

transform2 = transforms.ToTensor()

C:\Users\Yorick He\AppData\Roaming\Python\Python38\site-packages\torchvision\transforms\transforms.py:1435: UserWarning: The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. Please use 'fill' instead.
  warnings.warn(


In [6]:
dataset = TinyImageNetDataset(data_dir+'/train', transform=transforms.Compose([transform1, transform2]))
labels = dataset.get_labels()

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
idx_tr, idx_val = next(sss.split(labels, labels))

dataset_tr = torch.utils.data.Subset(dataset, idx_tr)
dataset_val = torch.utils.data.Subset(dataset, idx_val)

dataloader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=batch_size, shuffle=True, num_workers=workers)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers=workers)

print('Training Dataset Length: ' + str(len(dataset_tr)))
print('Validation Dataset Length: ' + str(len(dataset_val)))

class_to_idx = dataset.get_class_to_idx()
dataset_test = TinyImageNetTestDataset(data_dir+'/val', annot_filename = 'val_annotations.txt', class_to_idx = class_to_idx, transform=transform2)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=workers)

print('Test Dataset Length: ' + str(len(dataset_test)))

Training Dataset Length: 90000
Validation Dataset Length: 10000
Test Dataset Length: 10000


In [7]:
# model = CNN()
model = ResNet()
model.apply(init_weights)
if gpu:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=2,
                                  threshold=0.0001, verbose=True, min_lr=1e-8)
criterion = nn.CrossEntropyLoss()

model_filename = 'mymodel.pth'

In [8]:
def train(dataloader, model, optimizer, criterion):
    batch = 0
    loss = 0.0
    correct = 0.0
    top5_correct=0.0

    model.train()    
    
    with tqdm(total=len(dataloader.dataset), desc='train', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for X, Y in dataloader:
            if gpu:
                X = X.cuda()
                Y = Y.cuda()
            optimizer.zero_grad()        
            logits = model(X)
            cur_loss = criterion(logits, Y)
            cur_loss.backward()
            optimizer.step()
            loss += cur_loss.item()
            pred = logits.argmax(dim = 1)
            correct += pred.eq(Y).sum()
            top5_correct += topaccuracy(logits, Y, topk=(5,))
#             print(correct,top5_correct)
            
            pbar.update(dataloader.batch_size)

    
    loss = loss / float(len(dataloader.dataset))
    accuracy = float(correct) / float(len(dataloader.dataset)) * 100
    top5acc = float(top5_correct) / float(len(dataloader.dataset)) * 100
    
    return loss, accuracy, top5acc

In [9]:
def validate(dataloader, model, criterion):
    
    loss = 0.0
    correct = 0.0
    top5_correct = 0.0
    model.eval()
    
    with tqdm(total=len(dataloader.dataset), desc='validate', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for X, Y in dataloader:
            if gpu:
                X = X.cuda()
                Y = Y.cuda()        
            logits = model(X)
            cur_loss = criterion(logits, Y)
            loss += cur_loss.item()
            pred = logits.argmax(dim = 1)
            correct += pred.eq(Y).sum()
            top5_correct += topaccuracy(logits, Y, topk=(5,))
            pbar.update(dataloader.batch_size)
            
    loss = loss / float(len(dataloader.dataset))
    accuracy = float(correct) / float(len(dataloader.dataset)) * 100
    top5acc = float(top5_correct) / float(len(dataloader.dataset)) * 100

    return loss, accuracy, top5acc

In [10]:
def test(model, dataloader):
    
    loss = 0.0
    correct = 0.0
    top5_correct = 0.0
    model.eval()
    
    with tqdm(total=len(dataloader.dataset), desc='test', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
        for X, Y in dataloader:
            if gpu:
                X = X.cuda()
                Y = Y.cuda()        
            logits = model(X)
            cur_loss = criterion(logits, Y)
            loss += cur_loss.item()
            pred = logits.argmax(dim = 1)
            correct += pred.eq(Y).sum()
            top5_correct += topaccuracy(logits, Y, topk=(5,))
            pbar.update(dataloader.batch_size)
            
    loss = loss / float(len(dataloader.dataset))
    accuracy = float(correct) / float(len(dataloader.dataset)) * 100
    top5acc = float(top5_correct) / float(len(dataloader.dataset)) * 100

    return pred, loss, accuracy, top5acc


In [11]:
def fit(dataloader_tr, dataloader_val, model, criterion, optimizer, max_epoch = 100, scheduler = None, filename = None, early_stop = True, patience = 10, resume = False):
    start_epoch = 0
    early_stop_counter = 0
    min_loss_val = 1e10    
    loss_tr_hist = []
    loss_val_hist = []
    accuracy_tr_hist = []
    accuracy_val_hist = []
    top5_accuracy_tr_hist = []
    top5_accuracy_val_hist = []

    if resume == True:
        if filename is None:
            print('Please Provide File Name to load model')
            return
        start_epoch, model, optimizer, scheduler, early_stop_counter, loss_tr_hist, loss_val_hist, accuracy_tr_hist, \
        accuracy_val_hist = load_model(filename, model, optimizer, scheduler, mode = 'train')
        
    early_stopping = EarlyStopping(patience=patience, verbose=True)    
    
    for epoch in range(start_epoch+1, max_epoch + 1):
        t0 = time.time()

        loss_tr, accuracy_tr, top5_acc_tr = train(dataloader_tr, model, optimizer, criterion)
        loss_tr_hist.append(loss_tr)
        accuracy_tr_hist.append(accuracy_tr)
        top5_accuracy_tr_hist.append(top5_acc_tr)

        loss_val, accuracy_val, top5_acc_val = validate(dataloader_val, model, criterion)
        loss_val_hist.append(loss_val)
        accuracy_val_hist.append(accuracy_val)
        top5_accuracy_val_hist.append(top5_acc_val)
        
        print('Epoch:',epoch)
        print('Train Loss: {0:7.4f}  |  Top1 Accuracy: {1:7.4f}  |  Top5 Accuracy: {2:7.4f}'.format(loss_tr, accuracy_tr, top5_acc_tr))
        print('Validate Loss: {0:7.4f}  |  Top1 Accuracy: {1:7.4f}  |  Top5 Accuracy: {2:7.4f}'.format(loss_val, accuracy_val, top5_acc_val))
        
        if scheduler is not None:
            scheduler.step(accuracy_val)
        if early_stop:
            early_stopping(accuracy_val, model)
            # 若满足 early stopping 要求
            if early_stopping.early_stop:
                print("Early stopping in epoch",epoch)
                # 结束模型训练
                break
    model.load_state_dict(torch.load('checkpoint.pt'))
        
    return loss_tr_hist, loss_val_hist, accuracy_tr_hist, accuracy_val_hist, top5_accuracy_tr_hist, top5_accuracy_val_hist, model

In [12]:
loss_tr_hist, loss_val_hist, accuracy_tr_hist, accuracy_val_hist, top5_accuracy_tr_hist, top5_accuracy_val_hist, model = fit(
    dataloader_tr, dataloader_val, model, criterion, optimizer, max_epoch = 100, \
    scheduler = scheduler, filename = None, early_stop = True, patience = 10, resume = False
            )

train: 90.1kB [01:20, 1.12kB/s]                                                                     
validate: 10.1kB [00:06, 1.55kB/s]                                                                  
train:   0%|                                                            | 0.00/90.0k [00:00<?, ?B/s]

Epoch: 1
Train Loss:  0.0407  |  Top1 Accuracy:  1.1744  |  Top5 Accuracy:  5.6144
Validate Loss:  0.0433  |  Top1 Accuracy:  1.2600  |  Top5 Accuracy:  5.2000
Validation acc increased (0.000000 --> 1.260000).  Saving model ...


train: 90.1kB [01:13, 1.22kB/s]                                                                     
validate: 10.1kB [00:05, 1.75kB/s]                                                                  
train:   0%|                                                            | 0.00/90.0k [00:00<?, ?B/s]

Epoch: 2
Train Loss:  0.0387  |  Top1 Accuracy:  2.4800  |  Top5 Accuracy: 10.5467
Validate Loss:  0.0386  |  Top1 Accuracy:  3.1600  |  Top5 Accuracy: 12.4000
Validation acc increased (1.260000 --> 3.160000).  Saving model ...


train: 90.1kB [01:08, 1.32kB/s]                                                                     
validate: 10.1kB [00:05, 1.82kB/s]                                                                  
train:   0%|                                                            | 0.00/90.0k [00:00<?, ?B/s]

...

Epoch: 49
Train Loss:  0.0268  |  Top1 Accuracy: 21.8278  |  Top5 Accuracy: 47.8256
Validate Loss:  0.0257  |  Top1 Accuracy: 26.5400  |  Top5 Accuracy: 53.8700
Epoch 00049: reducing learning rate of group 0 to 1.5000e-08.
EarlyStopping counter: 9 out of 10


train: 90.1kB [01:17, 1.16kB/s]                                                                     
validate: 10.1kB [00:08, 1.26kB/s]                                                                  
train:   0%|                                                            | 0.00/90.0k [00:00<?, ?B/s]

Epoch: 50
Train Loss:  0.0268  |  Top1 Accuracy: 21.9000  |  Top5 Accuracy: 47.9978
Validate Loss:  0.0255  |  Top1 Accuracy: 27.0800  |  Top5 Accuracy: 54.9200
Validation acc increased (27.080000 --> 27.080000).  Saving model ...


train:  79%|█████████████████████████████████████████▊           | 71.0k/90.0k [01:21<00:21, 870B/s]


KeyboardInterrupt: 

In [13]:
# model = CNN()
# model = load_model('checkpoint.pt', model, mode = 'test')
if gpu:
    model.cuda()
pred, loss_test, accuracy_test, top5acc = test(model, dataloader_test)
print('Test Loss: {0:7.4f}  |  Test Accuracy: {1:7.4f}  |  Test Top5Accuracy: {2:7.4f}'.format(loss_test, accuracy_test, top5acc))

test: 10.1kB [00:08, 1.19kB/s]                                                                      


Test Loss:  0.0200  |  Test Accuracy: 44.6900  |  Test Top5Accuracy: 70.7500


In [ ]:
plot(loss_tr_hist, loss_val_hist, accuracy_tr_hist, accuracy_val_hist, top5_accuracy_tr_hist, top5_accuracy_val_hist)